# SQLite Example from CSV

Library used for small and medium DB. Also for debugging and testing.


## Import data

In [3]:
import pandas as pd

df = pd.read_excel(r'LineDetails - 2019-11-21T153544.536.xlsx', skiprows=5)
df.head()


,Product Number,Product Description,Last Date of Support,Last Date of Sale,PAK/Serial Number,Parent PAK/Serial Number,Major/Minor,Instance Number,Parent Instance Number,Installed Base Status,End Customer GU,End Customer GU Name,End Customer ID,End Customer Name,End Customer Address,End Customer City,End Customer State,End Customer Province,End Customer County,End Customer Country,End Customer Zip/Postal Code,Subscription/Service SKU,Subscription/Service Description,Subscription/Service Level,Coverage Start Date,Coverage End Date,Contract Number,Contract Line Status,Covered Line Number,Potential Takeover,Host/MAC ID,Carton ID,Quantity,Contract Line Amount,Contract Bill to GU,Contract Bill to GU Name,Contract Bill to ID,Contract Bill to Name,Contract Bill to Address,Contract Bill to City,...,Item Type,Warranty Type,Warranty Status,Warranty End Date,Ship Date,End User GU,Reseller GU,Product Bill To ID,Product Bill to Name,Product Bill To Address,Product Bill To City,Product Bill To State,Product Bill To Province,Product Bill To County,Product Bill To Country,Product Bill To Postal Code,Product Ship to ID,Product Ship to Name,Product Ship to Address,Product Ship To City,Product Ship To State,Product Ship To Province,Product Ship To County,Product Ship To Country,Product Ship To Postal Code,Distributor Bill To ID,Distributor Bill To Name,Distributor Bill To GU Name,Distributor Bill To GU ID,Distributor Bill To Address,Distributor Bill To City,Distributor Bill To State,Distributor Bill To Province,Distributor Bill To County,Distributor Bill To Country,Distributor Bill To Postal Code,Cisco Capital Bill to ID,Smart Account / Virtual Account,Eligible for quoting,Reason for ineligibility
0,XFP-10GLR-OC192SR,Multirate XFP module for 10GBASE-LR and OC192 ...,2022-01-31,2017-01-31,AGA1624N56Z,AGA1624N56Z,Major,1035963519,1035963519,Latest-INSTALLED,6547768,AMERICA MOVIL SAB DE CV,1655003,UNINET SA DE CV,COLIMA 229 SUR COL SAN ANTONIO,SAN ANTONIO,NAY,NAY,NaN,MX,63159,SP-AR3-XFP10GSR,SP Base Support with HW AR Level 3,SPAR3,2019-04-17,2021-12-31,93104331.0,ACTIVE,1.993714,No,NaN,NaN,1,0.0,6547768.0,AMERICA MOVIL SAB DE CV,896369.0,UNINET SA DE CV,INSURGENTES SUR 3500 PISO 4 COLONIA PENA POBRE,TLALPAN,...,CARD,WARR-90-DAY-LTD-HW,Not Under Warranty,2013-04-13,2012-10-13 00:00:00,NaN,NaN,896369.0,UNINET SA DE CV,INSURGENTES SUR 3500 PISO 4 COLONIA PENA POBRE,TLALPAN,DF,NaN,NaN,MX,14060,402349861.0,UNINET S.A. DE C.V.,PERIFERICO SUR 3190 COL.JARDINES DEL PEDREGAL,MEXICO,DISTRITO FEDERAL,NaN,NaN,MX,01900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
1,CFP-100G-LR4,100GBASE-LR4 CFP Module,2024-09-30,2019-09-27,FIN16340077,FIN16340077,Major,1057028961,1057028961,Latest-INSTALLED,6547768,AMERICA MOVIL SAB DE CV,1650916,UNINET SA DE CV,PINO SUAREZ 393 ESQ MELCHOR OCAMPO COL CENTRO,ZAPOPAN,JAL,JAL,NaN,MX,45100,SP-AR3-CFP100GL,SP Base Support with HW AR Level 3,SPAR3,2019-04-17,2021-12-31,93104331.0,ACTIVE,1.101904,No,NaN,NaN,1,0.0,6547768.0,AMERICA MOVIL SAB DE CV,896369.0,UNINET SA DE CV,INSURGENTES SUR 3500 PISO 4 COLONIA PENA POBRE,TLALPAN,...,CARD,WARR-DEFAULT-90DAY,Not Under Warranty,2013-04-30,2012-10-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
2,X2-10GB-LR=,10GBASE-LR X2 Module,NaT,NaT,AGA1633X37F,AGA1633X37F,Major,1077365527,1077365527,Latest-INSTALLED,6547768,AMERICA MOVIL SAB DE CV,400425870,UNINET INC,2323 BRYAN ST FLOOR 14TH SUITE 1400,DALLAS,TX,TX,NaN,US,75201,NaN,SP Base Support with HW AR Level 3,SPAR3,2019-04-17,2021-12-31,93104331.0,ACTIVE,1.993745,No,NaN,NaN,1,0.0,6547768.0,AMERICA MOVIL SAB DE CV,896369.0,UNINET SA DE CV,INSURGENTES SUR 3500 PISO 4 COLONIA PENA POBRE,TLALPAN,...,CARD,WARR-90-DAY-LTD-HW,Not Under Warranty,2013-05-15,2012-11-15 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
3,X2-10GB-LR=,10GBASE-LR

https://elitedatascience.com/machine-learning-projects-for-beginners#stock-prices

In [4]:
for i, columns in enumerate(df.columns):
  print(i, columns)

0 Product Number
1 Product Description
2 Last Date of Support
3 Last Date of Sale
4 PAK/Serial Number
5 Parent PAK/Serial Number
6 Major/Minor
7 Instance Number
8 Parent Instance Number
9 Installed Base Status
10 End Customer GU
11 End Customer GU Name
12 End Customer ID
13 End Customer Name
14 End Customer Address
15 End Customer City
16 End Customer State
17 End Customer Province
18 End Customer County
19 End Customer Country
20 End Customer Zip/Postal Code
21 Subscription/Service SKU
22 Subscription/Service Description
23 Subscription/Service Level
24 Coverage Start Date
25 Coverage End Date
26 Contract Number
27 Contract Line Status
28 Covered Line Number
29 Potential Takeover
30 Host/MAC ID
31 Carton ID
32 Quantity
33 Contract Line Amount
34 Contract Bill to GU
35 Contract Bill to GU Name
36 Contract Bill to ID
37 Contract Bill to Name
38 Contract Bill to Address
39 Contract Bill to City
40 Contract Bill to State
41 Contract Bill to Province
42 Contract Bill to County
43 Contract 

In [5]:
ProdInfo = df.iloc[:,:3].drop_duplicates()
ProdInfo



,Product Number,Product Description,Last Date of Support
0,XFP-10GLR-OC192SR,Multirate XFP module for 10GBASE-LR and OC192 ...,2022-01-31
1,CFP-100G-LR4,100GBASE-LR4 CFP Module,2024-09-30
2,X2-10GB-LR=,10GBASE-LR X2 Module,NaT
4,CRS-PM-DC,CRS Modular DC Power Module,NaT
11,GLC-LH-SMD=,"1000BASE-LX/LH SFP transceiver module, MMF/SMF...",NaT
...,...,...,...
22973,PRP-3=,^^Cisco XR 12000 Performance Route Processor 3,2020-06-30
23844,PWR-C45-1400DC,C4500 1400W DC Triple Input SP Power Supply-da...,2025-10-31
24504,12410-SMSE-BUN,^GSR 12410 Serial-MSE Configurable Bundle,NaT
24820,12406E-SFC=,^Enhanced Switch Fabric Card for 12406,2019-02-28


In [0]:
items = df.iloc[:, [4,5, 7,8]].drop_duplicates()

In [7]:
adresses = df.iloc[:, 12:21].drop_duplicates()
adresses

,End Customer ID,End Customer Name,End Customer Address,End Customer City,End Customer State,End Customer Province,End Customer County,End Customer Country,End Customer Zip/Postal Code
0,1655003,UNINET SA DE CV,COLIMA 229 SUR COL SAN ANTONIO,SAN ANTONIO,NAY,NAY,NaN,MX,63159
1,1650916,UNINET SA DE CV,PINO SUAREZ 393 ESQ MELCHOR OCAMPO COL CENTRO,ZAPOPAN,JAL,JAL,NaN,MX,45100
2,400425870,UNINET INC,2323 BRYAN ST FLOOR 14TH SUITE 1400,DALLAS,TX,TX,NaN,US,75201
4,1002651451,UNINET S A DE C V,BLV EMILIANO ZAPATA 2050 PISO 3,CULIACAN,SIN,SIN,NaN,MX,80000
5,1002594187,UNINET,AV LOPEZ COTILLA NO 1424 ENTRE PROGRESO Y AV ...,GUADALAJARA,JAL,JAL,NaN,MX,44160
...,...,...,...,...,...,...,...,...,...
22543,1693481,TELMEX RCDT,LAGO CHAIREL 6113 ESQ NORTE 4,FRANCISCO I MADERO,DF,DF,NaN,MX,11480
23892,418438231,TELEFONOS DE MEXICO SAB DE CV,NEXTENGO 78 PLANTA BAJA SANTA APOLONIA NEXTEN...,AZCAPOTZALCO,DISTRITO FEDERAL,DISTRITO FEDERAL,NaN,MX,02790
24504,1003052020,CLARO,SIMOMOTO 115,SAO PAULO,SP,SP,NaN,BR,05347
25738,401007162,TELMEX RCDT,CALLE 5 EMILIANO ZAPATA NO 8081 ZONA CENTRO,TIJUANA CENTRO,BC,BC,NaN,MX,22000


In [8]:
contracts = df.iloc[:, 34:45].drop_duplicates()
contracts

,Contract Bill to GU,Contract Bill to GU Name,Contract Bill to ID,Contract Bill to Name,Contract Bill to Address,Contract Bill to City,Contract Bill to State,Contract Bill to Province,Contract Bill to County,Contract Bill to Country,Contract Bill to Zip/Postal Code
0,6547768.0,AMERICA MOVIL SAB DE CV,8.963690e+05,UNINET SA DE CV,INSURGENTES SUR 3500 PISO 4 COLONIA PENA POBRE,TLALPAN,DF,DF,NaN,MX,14060
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
811,9556430.0,DATATEC LTD,1.000000e+09,PROMONLOGICALIS TECNOLOGIA E PARTICIPACOES LTDA,AVENIDA TAMBORE 267 ANDAR 18,BARUERI,SP,SP,NaN,BR,06454000
812,6547768.0,AMERICA MOVIL SAB DE CV,4.013029e+08,EMBRATEL SA,AV PRESIDENTE VARGAS 1012 CENTRO DEPTO DE CO...,RIO DE JANEIRO,RIO DE JANEIRO,RIO DE JANEIRO,BRAZIL,BR,20071 910
957,6547768.0,AMERICA MOVIL SAB DE CV,4.033089e+08,TELMEX USA LLC,3350 SW 148TH AVENUE SUITE 400,MIRAMAR,FL,FL,BROWARD,US,33027
1014,6547768.0,AMERICA MOVIL SAB DE CV,8.057700e+05,"COMPANIA DOMINICANA DE TELEFONOS, S.A",EDIFICIO SUMINISTRO AVENIDA JOHN F KENNEDY KM...,SANTO DOMINGO,NaN,NaN,NaN,DO,NaN
1016,6547768.0,AMERICA MOVIL SAB DE CV,4.025500e+08,CONSORCIO ECUATORIANO DE TELECOMUNICACIONES SA,AV. FRANCISCO DE ORELLANA S/N Y ALBERTO BORGE...,GUAYAQUIL,NaN,NaN,NaN,EC,EC090112
1018,6547768.0,AMERICA MOVIL SAB DE CV,2.727501e+06,TELMEX COLOMBIA S A,CUADRA 7 NO 71-52 TORRE B PISO 18,BOGOTA,NaN,NaN,NaN,CO,NaN
1565,9556430.0,DATATEC LTD,4.187634e+08,LOGICALIS ARGENTINA SA,VIAMONTE 1639 BUENOS AIRES,CAPITAL FEDERAL,NaN,NaN,NaN,AR,1055
1754,3958057.0,CHARTER COMMUNICATIONS INC,4.078713e+08,CHARTER COMMUNICATIONS,CHARTER COMMUNICATIONS LLC ACCOUNTS PAYABLE P...,SAINT LOUIS,MO,MO,SAINT LOUIS CITY,US,63131


## Got info from file and divided in dif unique combinations to drop in different DB

Functions to manipulate the DB

In [9]:
def type_column(dataframe):
  list_columns = []
  for column in dataframe:
    dtype = df[column].dtype
    dic = {'object':'TEXT', 'datetime64[ns]':'DATETIME', 'int64':'INTEGER', 'float64':'FLOAT' }
    list_columns.append([str(column).replace(' ','').replace('/',''), dic[str(dtype)]])
  return list_columns
  

def create_table(conn, name_of_table, columns):
  try:
    with conn:
      cursor = conn.cursor()
      query = f"Create Table {name_of_table} ("
      for c in columns:
        query += f"{c[0]} {c[1]}, "
      query = query[:-2] + ')'
      cursor.execute(query)
  except Exception as e:
    print(e)
  
  
  
def get_column(conn, column_name, table):
  with conn:
    cursor = conn.cursor()
    column = f"`{column_name}`"
    cursor.execute(f"SELECT {column} FROM {table}")
    return [c[0] for c in cursor.fetchall()]

  
  
def get_rows(conn, value, column_name, table):
  with conn:
    cursor = conn.cursor()
    column = f"`{column_name}`"
    value = value.replace("'","")
    cursor.execute(f"SELECT * FROM {table} WHERE {column} LIKE '%{value}%'")
    return[list(c) for c in cursor.fetchall()]
    

# def add_row(conn, row, column_name):
#   with conn:
#     cursor = conn.cursor()
    

"SELECT * FROM adresses WHERE `End Customer NAME` LIKE '%UNINET%'"

Creating 2 SQL DATABASES Prod Info and Items is Customer and Contracts and Adresses is Company

In [0]:
import sqlite3

# Creation of the DBs
Customer_SQLconn = sqlite3.connect('Customer.db')        
Company_SQLconn = sqlite3.connect('Company.db')        



Creating tables in different DB

In [0]:
create_table(Customer_SQLconn, 'ProdInfo', type_column(ProdInfo) )
create_table(Company_SQLconn, 'adresses', type_column(adresses) )
create_table(Company_SQLconn, 'Contracts', type_column(contracts) )
create_table(Customer_SQLconn, 'items', type_column(items) )

## Inserting Values in DB

In [12]:
ProdInfo.to_sql('ProdInfo', Customer_SQLconn, if_exists='replace')
adresses.to_sql('adresses', Company_SQLconn, if_exists='replace')
contracts.to_sql('Contracts', Company_SQLconn, if_exists='replace')
items.to_sql('items', Customer_SQLconn, if_exists='replace' )

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


## Fetch from a DB

In [34]:
for i, c in enumerate(get_column(Company_SQLconn, 'End Customer ID', 'adresses')):
  if i%10==0:
    print()
  print(c, end=' ')




1655003 1650916 400425870 1002651451 1002594187 1705128 1000802 417466707 406511733 1702810 
416607362 1695894 1209275 1002177171 1209856 1002456384 1209210 1000308 1002483758 988448 
1699607 401617300 1210037 1208621 1654983 419404632 1654432 1209678 1208534 1208568 
412250810 402454288 401605480 401617530 1022968656 1000220 1209212 1002627404 400422210 1691702 
1650936 134742603 1697276 1654968 1693823 1002822170 403140032 403138289 1650836 402570387 
1652273 1002502824 1003473142 417466470 2184651 418496505 988490 1904874 988455 401136921 
403438195 2000347239 404827583 1650849 1000045619 405399743 1002486058 1209251 1003266000 138967002 
2184655 1002597241 1208257 402451467 1650821 401321893 1650846 1002813995 2000351617 1002463530 
1650904 418155048 1209239 1019388289 1002738015 1650881 988482 1025554509 402838025 2918769 
1692503 1002785605 1209213 1002665166 419409933 1692259 1650837 1655627 403198272 1000691 
2000351712 1000254 400976838 1002737461 1209195 1002689672 988503 10

In [17]:
for r in get_rows(Company_SQLconn, 'claro', 'End customer Name', 'adresses'):
  print(r)

[18323, 137435610, 'CLARO ARGENTINA', ' OLLEROS 2770', 'BUENOS AIRES', None, None, None, 'AR', '1426']
[24504, 1003052020, 'CLARO', ' SIMOMOTO 115', 'SAO PAULO', 'SP', 'SP', None, 'BR', '05347']
